In [1]:
import tensorflow as tf
from datetime import datetime

%load_ext tensorboard

In [2]:
class SimpleModule(tf.Module):

    def __init__(self, name=None):
        super().__init__(name=name)
        self.a_variable = tf.Variable(5, dtype=tf.dtypes.float32, name='train_me')
        self.non_trainable_variable = tf.Variable(
            5, trainable=False, 
            dtype=tf.dtypes.float32, 
            name='do_not_train_me'
        )

    def __call__(self, x):
        return self.a_variable * x + self.non_trainable_variable
        

In [3]:
simple_module = SimpleModule(name='Simple')
simple_module(tf.constant(5.0)).numpy()

30.0

In [4]:
print("Trainable variables:", simple_module.trainable_variables)
print("All variables:", simple_module.variables)

Trainable variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)
All variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>, <tf.Variable 'do_not_train_me:0' shape=() dtype=float32, numpy=5.0>)


In [5]:
class Dense(tf.Module):

    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(
            tf.random.normal([in_features, out_features], name='w')
        )
        self.b = tf.Variable(tf.ones([out_features]), name='b')

    def __call__(self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

In [6]:
class SequentialModule(tf.Module):

    def __init__(self, name=None):
        super().__init__(name=name)
        self.dense1 = Dense(in_features=3, out_features=3)
        self.dense2 = Dense(in_features=3, out_features=2)

    def __call__(self, x):
        x = self.dense1(x)
        return self.dense2(x)

In [7]:
mymodel = SequentialModule(name='mymodel')
mymodel([[1.0, 2.0, 3.0]]).numpy()

array([[0.       , 3.3653815]], dtype=float32)

In [8]:
print(mymodel.submodules)
print()
for vrs in mymodel.variables:
    print(vrs)

(<__main__.Dense object at 0x14e4eff60>, <__main__.Dense object at 0x14e4efb38>)

<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>
<tf.Variable 'Variable:0' shape=(3, 3) dtype=float32, numpy=
array([[ 0.33689612, -0.7213998 , -0.69536483],
       [ 0.4329436 , -1.0470401 ,  0.33147052],
       [ 0.0113384 , -0.20763868,  0.21036239]], dtype=float32)>
<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([1., 1.], dtype=float32)>
<tf.Variable 'Variable:0' shape=(3, 2) dtype=float32, numpy=
array([[-1.3474002 ,  0.32018778],
       [ 1.6944605 , -0.09411707],
       [ 0.13095284,  1.031603  ]], dtype=float32)>


In [9]:
class FlexibleDense(tf.Module):

    def __init__(self, out_features, name=None):
        super().__init__(name=name)
        self.is_built = False
        self.out_features = out_features

    def __call__(self, x):
        if not self.is_built:
            self.in_features = x.shape[-1]
            self.w = tf.Variable(
                tf.random.normal([self.in_features, self.out_features]), 
                name='w'
            )
            self.b = tf.Variable(tf.ones([self.out_features]), name='b')
            self.is_built = True

        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)



In [10]:
class MySequentialModule(tf.Module):

    def __init__(self, name=None):
        super().__init__(name=name)
        self.dense1 = FlexibleDense(out_features=3, name='dense1')
        self.dense2 = FlexibleDense(out_features=2, name='dense2')

    def __call__(self, x):
        x = self.dense1(x)
        return self.dense2(x)


In [11]:
mymodel2 = MySequentialModule(name='mymodel2')
print(mymodel2(tf.constant([[1.0, 2.0, 3.0, 4.0]])).numpy())

[[1.411684 2.481882]]


In [12]:
chkp_path = 'my_checkpoint'
checkpoint = tf.train.Checkpoint(model=mymodel2)
checkpoint.write(chkp_path)

'my_checkpoint'

In [13]:
tf.train.list_variables(chkp_path)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/dense1/b/.ATTRIBUTES/VARIABLE_VALUE', [3]),
 ('model/dense1/w/.ATTRIBUTES/VARIABLE_VALUE', [4, 3]),
 ('model/dense2/b/.ATTRIBUTES/VARIABLE_VALUE', [2]),
 ('model/dense2/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 2])]

In [14]:
mymodel3 = MySequentialModule(name='mymodel3')
new_checkpoint = tf.train.Checkpoint(model=mymodel3)
new_checkpoint.restore(chkp_path)


In [15]:
print(mymodel3(tf.constant([[1.0, 2.0, 3.0, 4.0]])).numpy())

[[1.411684 2.481882]]


In [16]:
class MySequentialModule_Graph(tf.Module):

    def __init__(self, name=None):
        super().__init__(name=name)
        self.dense1 = FlexibleDense(out_features=3, name='dense1')
        self.dense2 = FlexibleDense(out_features=2, name='dense2')

    @tf.function
    def __call__(self, x):
        x = self.dense1(x)
        return self.dense2(x)
        

In [17]:
mymodel4 = MySequentialModule_Graph(name='mymodel4')

In [18]:
print(mymodel4(tf.constant([[1.0, 2.0, 3.0, 4.0]])).numpy())

[[ 0.      25.40216]]


In [19]:
stamp = datetime.now().strftime('%Y%m%d-%H%M%S')
logdir = './logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

In [20]:
mymodel5 = MySequentialModule_Graph(name='mymodel5')

In [21]:
tf.summary.trace_on(graph=True)
tf.profiler.experimental.start(logdir)

In [22]:
z = print(mymodel5(tf.constant([[2.0, 2.0, 2.0]])).numpy())
with writer.as_default():
    tf.summary.trace_export(
        name='my_func_trace',
        step=0,
        profiler_outdir=logdir)

[[0.        4.0053625]]
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


In [23]:
#%tensorboard --logdir logs/func; #http://localhost:6006

In [24]:
tf.saved_model.save(mymodel5, "mymodel5_saved")

INFO:tensorflow:Assets written to: mymodel5_saved/assets


In [25]:
!ls -l mymodel5_saved/

total 24
drwxr-xr-x  2 y837577  staff     64 Apr 11 14:07 assets
-rw-r--r--  1 y837577  staff  10620 Apr 11 18:54 saved_model.pb
drwxr-xr-x  4 y837577  staff    128 Apr 11 18:54 variables


In [26]:
!ls -l mymodel5_saved/variables

total 16
-rw-r--r--  1 y837577  staff  402 Apr 11 18:54 variables.data-00000-of-00001
-rw-r--r--  1 y837577  staff  355 Apr 11 18:54 variables.index


In [27]:
class FlexibleDenseKeras(tf.keras.layers.Layer):

    def __init__(self, out_features, **kwargs):
        super().__init__(**kwargs)
        self.out_features = out_features

    def build(self, input_shape):
        self.w = tf.Variable(
            tf.random.normal([input_shape[-1], self.out_features]),
            name='w'
        )
        self.b = tf.Variable(tf.zeros([self.out_features]), name='b')

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
        

In [28]:
mymodel6 = FlexibleDenseKeras(out_features=3)

In [30]:
print(mymodel6(tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]).numpy())

[[ 9.673981    1.7248194  -2.528175  ]
 [25.119492    0.96183205 -7.2091017 ]]
